In [136]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from scipy.stats import entropy
import sys
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [138]:
data = load_breast_cancer()
X = data['data']
y = data['target']
features = data['feature_names']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [140]:
class Tree:
    def __init__(self, parent, X, y, isRegression=False):
        self.isRegression = isRegression
        self.parent = parent
        self.X = X
        self.y = y
        self.size = len(self.X)
        self.prediction = np.mean(y) # the mean works for both classification & regression
        self.fitness = self._getFitness()
        self.leftChild, self.rightChild = None, None
#         print self.__repr__()
    
    def __repr__(self):
        return "size: %d, prediction: %.2f, fitness: %.2f" % (self.size, self.prediction, self.fitness)
    
    ''' called by the Tree fn to get its own fitness '''
    def _getFitness(self, y=None):
        if y is None:
            y = self.y
        if not self.isRegression:
            pos = np.mean(y)
            enpy = entropy([pos, 1 - pos]) # val between [0, 0.693), corresponding to H([0, 1]) & H([0.5, 0.5])
            return enpy
        else:
            return np.var(y) # variance of y(s) in a split        

    ''' figure out how good a potential split is '''
    def _getSplitFitness(self, splitIndex, splitVal):
        leftIndices, rightIndices = self.X[:,splitIndex] <= splitVal, self.X[:,splitIndex] > splitVal
        leftVal = self._getFitness(self.y[leftIndices]) 
        rightVal = self._getFitness(self.y[rightIndices])
        return leftVal + rightVal, leftIndices, rightIndices

    def split(self): # given the X and y, find split
        if self.fitness < 0.001 or self.size <= 20: # pseudo-recursion base case
            return

        potentialSplits = range(self.X.shape[1])
        bestFitness = sys.maxint
        bestSplitIndex, bestSplitVal, bestLeftIndices, bestRightIndices = None, None, None, None
        for splitIndex in potentialSplits: # loop through all features
            splitVals = np.unique(self.X[:,splitIndex])
            for splitVal in splitVals: # loop through all vals for the feature
                fitness, leftIndices, rightIndices = self._getSplitFitness(splitIndex, splitVal)
                if np.sum(leftIndices) <= 5 or np.sum(rightIndices) <= 5: # min_size for leaf
                    continue 
                if fitness < bestFitness: # small = good, for entropy or variance
                    bestFitness, bestSplitIndex, bestSplitVal = fitness, splitIndex, splitVal
                    bestLeftIndices, bestRightIndices = leftIndices, rightIndices
        if bestFitness is None or bestFitness == sys.maxint: return
        
        self.splitFitness, self.splitIndex, self.splitVal = bestFitness, bestSplitIndex, bestSplitVal
        
        leftChild = Tree(self, self.X[bestLeftIndices], self.y[bestLeftIndices], self.isRegression)
        rightChild = Tree(self, self.X[bestRightIndices], self.y[bestRightIndices], self.isRegression)
        self.leftChild, self.rightChild = leftChild, rightChild
        self.leftChild.split()
        self.rightChild.split()
    
    def predict(self, x):
        if self.leftChild is None and self.rightChild is None:
            return self.prediction
        if x[self.splitIndex] <= self.splitVal:
            return self.leftChild.predict(x)
        else:
            return self.rightChild.predict(x)
        

class DecisionTree:
    def __init__(self, isRegression=False):
        self.isRegression = isRegression
    
    def fit(self, X, y):
        tree = Tree(None, X, y, self.isRegression)
        tree.split()
        self.tree = tree
        
    def predict(self, X):
        return np.array([self.tree.predict(X[i]) for i in range(X.shape[0])])

In [141]:
clf = DecisionTree()
clf.fit(X_train, y_train)

In [142]:
# example of how these trees look like

print clf.tree
print "\n\n"
print clf.tree.leftChild
print clf.tree.rightChild

size: 381, prediction: 0.62, fitness: 0.66



size: 279, prediction: 0.85, fitness: 0.43
size: 102, prediction: 0.00, fitness: 0.00


In [143]:
print "train acc: %.2f" % (accuracy_score(y_train, (clf.predict(X_train) > 0.5) * 1))
print "test acc: %.2f" % (accuracy_score(y_test, (clf.predict(X_test) > 0.5) * 1))

train acc: 0.98
test acc: 0.95
